In [1]:
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.applications.xception import Xception
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
import sklearn.metrics as metrics

import numpy as np
import pandas as pd

import pickle
import sys, gc, os
import glob

import matplotlib.pyplot as plt
import seaborn as sns


Using TensorFlow backend.


In [2]:
d=128

In [3]:
# https://keras.io/examples/cifar10_resnet/
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


In [4]:
le = LabelEncoder()

train = np.empty(shape=(0,d,d,3), dtype=np.int8)
labels = np.empty(shape=(0,2), dtype=np.float)

for f in glob.glob("../data/train_Xception_*"):
    print(f)
    train_loader = np.load(f)
    fakes = train_loader['fakes']
    train = np.append(train,fakes,axis=0)
    print('fakes',len(fakes))
    reals = train_loader['reals']
    train = np.append(train,reals,axis=0)
    print('reals',len(reals))
    l = np.append(np.full(len(fakes),'FAKE'),np.full(len(reals),'REAL'))
    l = le.fit_transform(l)
    l = np_utils.to_categorical(l, 2)
    print('labels',labels.ndim,l.ndim)
    labels = np.append(labels,l,axis=0)
    
print('train=',len(train))
print('labels',len(labels))

../data/train_Xception_3.npz
fakes 435
reals 438
labels 2 2
../data/train_Xception_2.npz
fakes 440
reals 433
labels 2 2
../data/train_Xception_1.npz
fakes 449
reals 443
labels 2 2
../data/train_Xception_4.npz
fakes 447
reals 446
labels 2 2
train= 3531
labels 3531


In [5]:
# Input image dimensions.
input_shape = train[1].shape
print(input_shape)
num_classes=2
# Training parameters
batch_size = 32  # orig paper trained all networks with batch_size=128
epochs = 200



(128, 128, 3)


In [6]:
model = Xception(include_top=True, weights=None, input_tensor=None, input_shape=input_shape, pooling=None, classes=num_classes)


model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])
model.summary()

# Prepare model model saving directory.
save_dir = '../models'
model_name = 'DDC_Xception_model.{epoch:03d}.h5' 
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=0.2,
                               cooldown=0,
                               patience=2,
                               min_lr=0.5e-6)

earlystop = EarlyStopping(monitor='val_loss',
                          patience=6)

callbacks = [checkpoint, lr_reducer, lr_scheduler, earlystop]


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 63, 63, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 63, 63, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 63, 63, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [7]:
(trainX, testX, trainY, testY) = train_test_split(train, labels, test_size=0.25, random_state=42)

In [8]:
del train, labels

In [9]:
history = model.fit(trainX,
              trainY,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(testX, testY),
              shuffle=True,
              callbacks=callbacks)

Train on 2648 samples, validate on 883 samples
Epoch 1/200
Learning rate:  0.001
2648/2648 [==============================] - 610s 231ms/step - loss: 0.8399 - acc: 0.5019 - val_loss: 1.0436 - val_acc: 0.5142

Epoch 00001: val_loss improved from inf to 1.04361, saving model to ../models/DDC_Xception_model.001.h5
Epoch 2/200
Learning rate:  0.001
2648/2648 [==============================] - 600s 226ms/step - loss: 0.7116 - acc: 0.5302 - val_loss: 0.9927 - val_acc: 0.4904

Epoch 00002: val_loss improved from 1.04361 to 0.99272, saving model to ../models/DDC_Xception_model.002.h5
Epoch 3/200
Learning rate:  0.001
2648/2648 [==============================] - 580s 219ms/step - loss: 0.7072 - acc: 0.5661 - val_loss: 0.8542 - val_acc: 0.4972

Epoch 00003: val_loss improved from 0.99272 to 0.85420, saving model to ../models/DDC_Xception_model.003.h5
Epoch 4/200
Learning rate:  0.001
2648/2648 [==============================] - 590s 223ms/step - loss: 0.6846 - acc: 0.5691 - val_loss: 0.8487 - va

KeyboardInterrupt: 

In [ ]:
f, (ax1, ax2) = plt.subplots(nrows = 1, ncols = 2, figsize=(15, 8))

# Plot training & validation accuracy values
ax1.plot(history.history['acc'])
ax1.plot(history.history['val_acc'])
ax1.set_title('Model accuracy')
ax1.set(xlabel="Epoch", ylabel="Accuracy")
ax1.legend(['Train', 'Test'], loc='down right')

# Plot training & validation loss values
ax2.plot(history.history['loss'])
ax2.plot(history.history['val_loss'])
ax2.set_title('Model loss')
ax2.set(xlabel="Epoch", ylabel="Loss")
ax2.legend(['Train', 'Test'], loc='upper right')

plt.show()

In [ ]:
predictions = model.predict(faces)

In [ ]:
metrics.log_loss(labels[:,0],predictions[:,0])